# IMPORTS

In [1]:
import os
import pandas as pd
import time
import random
import getpass
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from bs4 import BeautifulSoup
from tqdm import tqdm

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

# INICIAR EL DRIVER Y LOGGEANDO LINKEDIN

In [3]:
def driver():
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument(f'user-agent={user_agent}')
    options.add_argument("--window-size=1920,1080")
    return webdriver.Chrome(service=s, options=options)

In [4]:
def linkedin_login():
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    login_button = driver.find_element(By.CLASS_NAME,'sign-in-form__submit-button')
    login_button.click()


# DA POSITIONS

In [5]:
def data_analyst():
    #Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36'
    s = Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument('--incognito')
    options.add_argument(f'user-agent={user_agent}')
    options.binary_location = '/Applications/Google Chrome.app/Contents/MacOS/Google Chrome'
    driver = webdriver.Chrome(service=s, options=options)
    mail = 'cocazapata.21@gmail.com'
    pw = 'fcbarcelona21'
    driver.get('https://www.linkedin.com')
    username = driver.find_element(By.ID, 'session_key').send_keys(mail)
    password = driver.find_element(By.ID, 'session_password').send_keys(pw)
    #login_button = driver.find_element(By.CLASS_NAME,'btn-md btn-primary flex-shrink-0 cursor-pointer sign-in-form__submit-btn--full-width
    
    titles = []
    companies = []
    locations = []
    modalities = []
    descriptions = []
    #for i in tqdm(range(0, 976, 25)):
    for i in tqdm(range(0, 2, 1)):

        driver.get(f'''https://www.linkedin.com/jobs/search/?currentJobId=3642678730&geoId=107025191&keywords=data%20analyst&location=Barcelona%2C%20Catalonia%2C%20Spain&refresh=true={i}''')
        time.sleep(random.randint(3, 5))
        page_source = driver.page_source
        
        soup = BeautifulSoup(page_source, "html.parser")
        print(soup)
        all_listings = []
        keep_scrolling = True

        while keep_scrolling:
            listings = driver.find_elements(By.CSS_SELECTOR,".job-card-list__title")
            if set(listings) == set(all_listings):
                keep_scrolling = False
            else:
                new_listings = list(set(listings) - set(all_listings))
                print(len(new_listings))
                for listing in new_listings:
                    all_listings.append(listing)
                    try:
                        driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth'});", listing)
                        time.sleep(random.randint(1, 3))
                        listing.click()
                        time.sleep(random.randint(2, 5))
                        page_source = driver.page_source
                        soup = BeautifulSoup(page_source, "html.parser")
                        try:
                            title = driver.find_element(By.XPATH, '//*[@id="ember1195"]/h2').text
                            #div_element = driver.find_element_by_class_name("jobs-unified-top-card__content--two-pane")
                            print(div_element)
                            #title = div_element.find_element_by_tag_name("h2").text
                            
                            
                            if 'data' in title.lower() or 'datos' in title.lower():
                                titles.append(title)
                                try:
                                    company = driver.find_element(By.XPATH, "//*[contains(@class, 'ember-view t-black t-normal')]").text
                                    companies.append(company)
                                except NoSuchElementException:
                                    companies.append('Unknown')
                                    #driver.save_screenshot("screenshot.png")
                                try:
                                    location = soup.select('body > div.application-outlet > div.authentication-outlet > div.job-search-ext > div.jobs-search-two-pane__wrapper > div > section.jobs-search__right-rail > div > div > div:nth-child(1) > div > div:nth-child(1) > div > div.jobs-unified-top-card__content--two-pane > div.mt2 > span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__bullet')[0].get_text().strip()
                                    locations.append(location)
                                except IndexError:
                                    locations.append('Unknown')
                                    #print('location error')
                                try:
                                    modality = soup.select('span.jobs-unified-top-card__subtitle-primary-grouping.mr2.t-black > span.jobs-unified-top-card__workplace-type')[0].get_text()
                                    modalities.append(modality)
                                except IndexError:
                                    modalities.append('Unknown')
                                    #print('modality error')
                                try:
                                    description = soup.select('#job-details > span')[0].get_text()
                                    descriptions.append(description)
                                except IndexError:
                                    descriptions.append('Unknown')
                                    #print('description error')
                        except NoSuchElementException:
                            driver.save_screenshot(f"{random.randint(0, 100)}.png")
                    except StaleElementReferenceException:
                        driver.save_screenshot(f"{random.randint(0, 100)}.png")
                time.sleep(random.randint(1, 2))
    driver.quit()
    df = pd.DataFrame({'title': titles, 'company': companies, 'location': locations,
                  'modality': modalities, 'description': descriptions})
    return df

In [6]:
analyst = data_analyst()
analyst

 50%|██████████████████████▌                      | 1/2 [00:10<00:10, 10.30s/it]

<html lang="es"><head>
<meta content="d_jobs_guest_search" name="pageKey"/>
<meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
<meta content="es_ES" name="locale"/>
<meta data-app-version="2.0.1312" data-browser-id="be40c3dc-0aeb-4d79-8c8d-36e4bf0039af" data-call-tree-id="AAX/ahbDfcqjmIV7EYVbMw==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;zWiJPVyzQXuyWhE7tRPijA==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://es.linkedin.com/jobs/data-analyst-empleos-barcelona" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<!-- -->
<!-- -->
<link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
<script>
          function getDfd() {let yFn,nFn;const p=new Promise(function(y, n){yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}
          wind

100%|█████████████████████████████████████████████| 2/2 [00:15<00:00,  7.96s/it]

<html lang="es"><head>
<meta content="d_jobs_guest_search" name="pageKey"/>
<meta content="urlType=jserp_custom;emptyResult=false" name="linkedin:pageTag"/>
<meta content="es_ES" name="locale"/>
<meta data-app-version="2.0.1312" data-browser-id="be40c3dc-0aeb-4d79-8c8d-36e4bf0039af" data-call-tree-id="AAX/ahdXx+QIH6HMFnS+yw==" data-disable-jsbeacon-pagekey-suffix="false" data-enable-page-view-heartbeat-tracking="" data-member-id="0" data-multiproduct-name="jobs-guest-frontend" data-page-instance="urn:li:page:d_jobs_guest_search;41y8bgMETfysbwerGEsUsQ==" data-service-name="jobs-guest-frontend" id="config"/>
<link href="https://es.linkedin.com/jobs/data-analyst-empleos-barcelona" rel="canonical"/>
<!-- --><!-- -->
<!-- -->
<!-- -->
<!-- -->
<!-- -->
<link href="https://static.licdn.com/aero-v1/sc/h/al2o9zrvru7aqj8e1x2rzsrca" rel="icon"/>
<script>
          function getDfd() {let yFn,nFn;const p=new Promise(function(y, n){yFn=y;nFn=n;});p.resolve=yFn;p.reject=nFn;return p;}
          wind

,title,company,location,modality,description
